<a href="https://colab.research.google.com/github/shipley7/fifa/blob/main/FIFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv("combined_match_data.csv") # getting the data
df.head()

,index,Player,#,Nation,Pos,Age,Min,Gls,Ast,PK,...,AvgLen.1,Opp,Stp,Stp%,#OPA,AvgDist,home,game_id,top100,source_file
0,0,14 Players,NaN,NaN,NaN,NaN,990.0,3.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,bundesliga_2017-2018_player_data.csv
1,1,Arjen Robben,10.0,nl NED,RW,33-207,30.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,bundesliga_2017-2018_player_data.csv
2,2,Arturo Vidal,23.0,cl CHI,LM,30-088,90.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,bundesliga_2017-2018_player_data.csv
3,3,Corentin Tolisso,24.0,fr FRA,RM,23-015,90.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,bundesliga_2017-2018_player_data.csv
4,4,David Alaba,27.0,at AUT,LB,25-055,90.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,bundesliga_2017-2018_player_data.csv


In [3]:
df = df[df["Player"].str.contains("Players")==False] # removing excess rows that have no player information
df = df.drop(['index', "#"], axis = 1) # index and column have no impact on anything
df[["Age Years", "Age Days"]] = (df.Age.str.split("-", expand = True)).apply(pd.to_numeric) # splitting age column into two other columns
df[["league", "season"]] = df["source_file"].replace("_player_data.csv", "", regex = True).str.split("_", expand = True) # splitting the source file into a league and season
df["season"] = (df["season"].str[:4].astype(int)) + 1 # season will be an integer representing the year the season ended
df = pd.get_dummies(df, columns=["league"]) # league will be a boolean variable representing which leage we are using
df = df.drop(['Age','source_file', "game_id"], axis = 1) # removing age, source file, and game id as they are no longer needed
df = df.fillna(0) # filling any NA with 0
df

,Player,Nation,Pos,Min,Gls,Ast,PK,PKatt,Sh,SoT,...,home,top100,Age Years,Age Days,season,league_bundesliga,league_la-liga,league_ligue-1,league_premier-league,league_serie-a
1,Arjen Robben,nl NED,RW,30.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1,0,33.0,207.0,2018,True,False,False,False,False
2,Arturo Vidal,cl CHI,LM,90.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1,0,30.0,88.0,2018,True,False,False,False,False
3,Corentin Tolisso,fr FRA,RM,90.0,1.0,0.0,0.0,0.0,4.0,3.0,...,1,0,23.0,15.0,2018,True,False,False,False,False
4,David Alaba,at AUT,LB,90.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,25.0,55.0,2018,True,False,False,False,False
5,Franck Ribéry,fr FRA,LW,76.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,34.0,133.0,2018,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452035,Ondrej Duda,sk SVK,DM,90.0,0.0,1.0,0.0,0.0,2.0,0.0,...,0,0,30.0,171.0,2025,False,False,False,False,True
452036,Paweł Dawidowicz,pl POL,DM,90.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,30.0,5.0,2025,False,False,False,False,True
452037,Simone Perilli,it ITA,GK,90.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,30.0,138.0,2025,False,False,False,False,True
452038,Suat Serdar,de GER,DM,80.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0,0,28.0,44.0,2025,False,False,False,False,True


In [4]:
# splitting into keepers and outfielders since they all have different statistics
keepers = df[df["Pos"] == "GK"]
outfielders = df[df["Pos"] != "GK"]

In [5]:
# for keepers, removing everything that a keeper doesn't normally do
keepers = keepers.drop(["Pos", 'Gls', 'Ast', 'PK', 'PKatt', 'Sh',
       'SoT', 'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG',
       'xAG', 'SCA', 'GCA', 'Cmp_x', 'Att_x', 'Cmp%_x', 'PrgP', 'Carries',
       'PrgC', 'Att_x.1', 'Succ'], axis = 1)
keepers

,Player,Nation,Min,CrdY,CrdR,SoTA,GA,Saves,Save%,PSxG,...,home,top100,Age Years,Age Days,season,league_bundesliga,league_la-liga,league_ligue-1,league_premier-league,league_serie-a
13,Sven Ulreich,de GER,90.0,0.0,0.0,4.0,1.0,3.0,75.0,0.9,...,1,0,29.0,15.0,2018,True,False,False,False,False
19,Bernd Leno,de GER,90.0,0.0,0.0,8.0,3.0,5.0,75.0,3.5,...,0,0,25.0,167.0,2018,True,False,False,False,False
36,Koen Casteels,be BEL,90.0,0.0,0.0,5.0,3.0,2.0,40.0,1.9,...,1,0,25.0,55.0,2018,True,False,False,False,False
56,Roman Bürki,ch SUI,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,26.0,278.0,2018,True,False,False,False,False
70,Rune Jarstein,no NOR,90.0,0.0,0.0,2.0,0.0,2.0,100.0,0.1,...,1,0,32.0,324.0,2018,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451966,Michele Di Gregorio,it ITA,90.0,0.0,0.0,5.0,2.0,3.0,60.0,1.4,...,0,0,27.0,302.0,2025,False,False,False,False,True
451981,Marco Carnesecchi,it ITA,90.0,0.0,0.0,10.0,3.0,7.0,70.0,3.0,...,1,0,24.0,328.0,2025,False,False,False,False,True
452006,Zion Suzuki,jp JPN,90.0,0.0,0.0,4.0,2.0,2.0,50.0,1.9,...,0,0,22.0,277.0,2025,False,False,False,False,True
452010,Devis Vásquez,co COL,90.0,0.0,0.0,2.0,2.0,0.0,0.0,1.5,...,1,0,27.0,13.0,2025,False,False,False,False,True


In [6]:
position_dummies = outfielders.Pos.str.get_dummies(sep = ",") # since most players play mutliple positions in a match, we will have to multi-hot encode
outfielders = pd.concat([outfielders.drop(columns=["Pos"]), position_dummies], axis = 1) # combining our binary position columns with the remaining dataframe
outfielders[['AM', 'CB', 'CM', 'DF', 'DM',
       'FW', 'LB', 'LM', 'LW', 'MF', 'RB', 'RM', 'RW', 'WB']] = outfielders[['AM', 'CB', 'CM', 'DF', 'DM',
       'FW', 'LB', 'LM', 'LW', 'MF', 'RB', 'RM', 'RW', 'WB']].astype('bool') # change position to boolean (did they play this position in the match or not?)
outfielders = outfielders.drop(['SoTA', 'GA', 'Saves', 'Save%', 'PSxG',
       'Cmp_y', 'Att_y', 'Cmp%_y', 'Att (GK)', 'Thr', 'Launch%', 'AvgLen',
       'Att_y.1', 'Launch%.1', 'AvgLen.1', 'Opp', 'Stp', 'Stp%', '#OPA',
       'AvgDist', 'GK'], axis = 1) # removing keeper statistics
outfielders

,Player,Nation,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,...,DM,FW,LB,LM,LW,MF,RB,RM,RW,WB
1,Arjen Robben,nl NED,30.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,Arturo Vidal,cl CHI,90.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,False,False,False,True,False,False,False,False,False,False
3,Corentin Tolisso,fr FRA,90.0,1.0,0.0,0.0,0.0,4.0,3.0,0.0,...,False,False,False,False,False,False,False,True,False,False
4,David Alaba,at AUT,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,True,False,False,False,False,False,False,False
5,Franck Ribéry,fr FRA,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452034,Nicolás Valentini,ar ARG,90.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
452035,Ondrej Duda,sk SVK,90.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,...,True,False,False,False,False,False,False,False,False,False
452036,Paweł Dawidowicz,pl POL,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,False,False,False,False,False,False,False,False,False
452038,Suat Serdar,de GER,80.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,True,False,False,False,False,False,False,False,False,False


In [ ]:
# # here, I am trying to see if a model could accuartely predict the positions one would play
# pos_prediction = outfielders.drop(["Player", "Nation"], axis = 1) # we cannot use Player, since that is a dead giveaway, and nation is useless

# # since players only get better or worse with time, our testing data must come after our training data
# train_seasons = [2018, 2019, 2020, 2021]
# test_season = [2022]
# train_df = pos_prediction[pos_prediction['season'].isin(train_seasons)]
# test_df = pos_prediction[pos_prediction['season'].isin(test_season)]

# positions = ['AM', 'CB', 'CM', 'DF', 'DM',
#        'FW', 'LB', 'LM', 'LW', 'MF', 'RB', 'RM', 'RW', 'WB'] # our target variables

# X_train = train_df.drop(columns = positions)
# y_train = train_df[positions]
# X_test = test_df.drop(columns = positions)
# y_test = test_df[positions]

# model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state = 42, class_weight='balanced')) # ChatGPT recommended this
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

# print(classification_report(y_test, y_pred, target_names=positions)) # results (not very good, but can be expected since there are 14 possible positions and not necessarily one right answer)

# # Going to try again but this time will generalize positions so maybe we will only have FW, MID, DEF


              precision    recall  f1-score   support

          AM       0.33      0.00      0.00      3534
          CB       0.82      0.55      0.66     10047
          CM       0.50      0.01      0.02      8623
          DF       0.00      0.00      0.00         0
          DM       0.00      0.00      0.00      3282
          FW       0.72      0.44      0.54      8962
          LB       0.46      0.01      0.02      3689
          LM       0.00      0.00      0.00      3503
          LW       0.22      0.00      0.00      3061
          MF       0.00      0.00      0.00         0
          RB       0.49      0.00      0.01      3785
          RM       0.67      0.00      0.00      3476
          RW       0.00      0.00      0.00      3051
          WB       0.33      0.00      0.00      2730

   micro avg       0.77      0.17      0.27     57743
   macro avg       0.32      0.07      0.09     57743
weighted avg       0.48      0.17      0.20     57743
 samples avg       0.19   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/me

In [ ]:
# # Let's generalize into FWD, MID, and DEF

# pos_prediction["Pos_Gen_FWD"] = np.where(pos_prediction['FW'] | pos_prediction['LW'] | pos_prediction['RW'], True, False)

# pos_prediction["Pos_Gen_MID"] = np.where(pos_prediction['AM'] | pos_prediction['CM'] | pos_prediction['DM'] | pos_prediction['LM'] | pos_prediction['MF'] | pos_prediction['RM'], True, False)

# pos_prediction["Pos_Gen_DEF"] = np.where(pos_prediction['CB'] | pos_prediction['DF'] | pos_prediction['LB'] | pos_prediction['RB'] | pos_prediction['WB'], True, False)

In [ ]:
# pos_prediction = pos_prediction.drop(positions, axis = 1) # we cannot use Player, since that is a dead giveaway, and nation is useless

# # since players only get better or worse with time, our testing data must come after our training data
# train_seasons = [2018, 2019, 2020, 2021]
# test_season = [2022]
# train_df = pos_prediction[pos_prediction['season'].isin(train_seasons)]
# test_df = pos_prediction[pos_prediction['season'].isin(test_season)]

# positions = ["Pos_Gen_FWD", "Pos_Gen_MID", "Pos_Gen_DEF"] # Let's generalize

# # Same as before
# X_train = train_df.drop(columns = positions)
# y_train = train_df[positions]
# X_test = test_df.drop(columns = positions)
# y_test = test_df[positions]

# model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state = 42, class_weight='balanced')) # ChatGPT recommended this
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

# print(classification_report(y_test, y_pred, target_names=positions)) # looks better

              precision    recall  f1-score   support

 Pos_Gen_FWD       0.71      0.60      0.65     14378
 Pos_Gen_MID       0.62      0.48      0.54     20192
 Pos_Gen_DEF       0.81      0.62      0.70     19370

   micro avg       0.71      0.56      0.63     53940
   macro avg       0.71      0.57      0.63     53940
weighted avg       0.71      0.56      0.63     53940
 samples avg       0.57      0.57      0.57     53940



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
selected_season = int(input("Please enter the season you would like to investigate:"))

top_100_outfielders = outfielders[(outfielders['top100'] != 0) & (outfielders['season'] == selected_season)] # get players who at some point have been in the top 100
remainders = outfielders[(outfielders["top100"] == 0) & (outfielders['season'] == selected_season)] # players who are not in the top 100

top_100_outfielders_agg = top_100_outfielders.groupby("Player").mean(numeric_only=True).reset_index() # aggregate each top 100 player's stats (mean) into one row
remainders_agg = remainders.groupby("Player").mean(numeric_only=True).reset_index() # aggreate remainding players stats into one row

# Should we keep leagues? 'league_bundesliga', 'league_la-liga', 'league_ligue-1', 'league_premier-league', 'league_serie-a' For now we will
top_agg = top_100_outfielders_agg.drop(["Player", "home", "Age Years", "Age Days", "top100", "season", 'league_bundesliga', 'league_la-liga', 'league_ligue-1', 'league_premier-league', 'league_serie-a'], axis = 1) # removing columns I do not believe will help
rest_agg = remainders_agg.drop(["Player", "home", "Age Years", "Age Days", "top100", "season", 'league_bundesliga', 'league_la-liga', 'league_ligue-1', 'league_premier-league', 'league_serie-a'], axis = 1) # removing columns I do not believe will help

# We should standardize our data
scaler = StandardScaler()
top_scaled_agg = scaler.fit_transform(top_agg)
rest_scaled_agg = scaler.fit_transform(rest_agg)

knn_out_agg = NearestNeighbors(n_neighbors=5, metric="euclidean") # KNN
knn_out_agg.fit(top_scaled_agg) # Fit with the top 100 player data

distances, indices = knn_out_agg.kneighbors(rest_scaled_agg) # for all non-top 100 players, find their neighbors (who will be top 100)
player = input("Please enter the name of an outfielder:") # Let's add player input for fun
player_number = remainders_agg[remainders_agg["Player"] == player].index[0] # Find the row where the player is
print("Candidate:", remainders_agg.iloc[player_number]['Player']) # Ensuring we have the right player
print("Similar top 100 players:")

# The top five top 100 players most similar to this rando for that year
for i, number in enumerate(indices[player_number]):
  print(f"{i+1}. {top_100_outfielders_agg.iloc[number]['Player']}")

Please enter the season you would like to investigate:2018
Please enter the name of an outfielder:Anthony Martial
Candidate: Anthony Martial
Similar top 100 players:
1. Memphis
2. Lorenzo Insigne
3. Leroy Sané
4. Sadio Mané
5. Philippe Coutinho


In [24]:
# We will now do the same as above, but this time we will enter a top 100 player
# Then, we find the top five non-top 100 players who are most similar according to KNN

knn_out_agg_2 = NearestNeighbors(n_neighbors=5, metric="euclidean")
knn_out_agg_2.fit(rest_scaled_agg)

distances, indices = knn_out_agg_2.kneighbors(top_scaled_agg)
player = input("Please enter the name of an outfielder:")
player_number = top_100_outfielders_agg[top_100_outfielders_agg["Player"] == player].index[0]
print("Candidate:", top_100_outfielders_agg.iloc[player_number]['Player'])
print("Similar normal 100 players:")

for i, number in enumerate(indices[player_number]):
  print(f"{i+1}. {remainders_agg.iloc[number]['Player']}")

Please enter the name of an outfielder:Aymeric Laporte
Candidate: Aymeric Laporte
Similar normal 100 players:
1. Andreas Christensen
2. Mateo Musacchio
3. Warmed Omari
4. Benoît Badiashile
5. Karol Mets


In [36]:
# Now, let's do the keepers
selected_season = int(input("Please enter the season you would like to investigate:"))

top_100_keepers = keepers[(keepers['top100'] != 0) & (keepers['season'] == selected_season)]
remainders = keepers[(keepers["top100"] == 0) & (keepers['season'] == selected_season)]

top_100_keepers_agg = top_100_keepers.groupby("Player").mean(numeric_only=True).reset_index()
remainders_agg = remainders.groupby("Player").mean(numeric_only=True).reset_index()

top_agg = top_100_keepers_agg.drop(["Player", "home", "Age Years", "Age Days", "top100", "season", 'league_bundesliga', 'league_la-liga', 'league_ligue-1', 'league_premier-league', 'league_serie-a'], axis = 1)
rest_agg = remainders_agg.drop(["Player", "home", "Age Years", "Age Days", "top100", "season", 'league_bundesliga', 'league_la-liga', 'league_ligue-1', 'league_premier-league', 'league_serie-a'], axis = 1)

scaler = StandardScaler()
top_scaled_agg = scaler.fit_transform(top_agg)
rest_scaled_agg = scaler.fit_transform(rest_agg)

knn_keep_agg = NearestNeighbors(n_neighbors=5, metric="euclidean")
knn_keep_agg.fit(top_scaled_agg)

distances, indices = knn_keep_agg.kneighbors(rest_scaled_agg)
player = input("Please enter the name of a goalkeeper:")
player_number = remainders_agg[remainders_agg["Player"] == player].index[0]
print("Candidate:", remainders_agg.iloc[player_number]['Player'])
print("Similar top 100 players:")

for i, number in enumerate(indices[player_number]):
  print(f"{i+1}. {top_100_keepers_agg.iloc[number]['Player']}")

Please enter the season you would like to investigate:2018
Please enter the name of a goalkeeper:Nick Pope
Candidate: Nick Pope
Similar top 100 players:
1. David de Gea
2. Jan Oblak
3. Jordan Pickford
4. Thibaut Courtois
5. Hugo Lloris


In [18]:
# "Reverse, reverse"

knn_keep_agg_2 = NearestNeighbors(n_neighbors=5, metric="euclidean")
knn_keep_agg_2.fit(rest_scaled_agg)

distances, indices = knn_keep_agg_2.kneighbors(top_scaled_agg)
player = input("Please enter the name of a goalkeeper:")
player_number = top_100_keepers_agg[top_100_keepers_agg["Player"] == player].index[0]
print("Candidate:", top_100_keepers_agg.iloc[player_number]['Player'])
print("Similar normal players:")

for i, number in enumerate(indices[player_number]):
  print(f"{i+1}. {remainders_agg.iloc[number]['Player']}")

Please enter the name of a goalkeeper:Gianluigi Donnarumma
Candidate: Gianluigi Donnarumma
Similar normal players:
1. Willy Caballero
2. Guillaume Restes
3. Yohann Pelé
4. Semuel Pizzignacco
5. Diego Conde


In [ ]:
# Here we will try to predict likelihood of a player being in the top 100 for 2025

In [ ]:
# Now let's try some logisitic regression to get predicted probabilities of becoming a future star

outfielders_agg = outfielders.groupby("Player").mean(numeric_only=True).reset_index() # Aggregating again
outfielders_agg = outfielders_agg.drop(["home", "Age Years", "Age Days", "season"], axis = 1) # Dropping non-essential columns

X = outfielders_agg.drop(["top_100", "Player"], axis = 1) # X data
y = outfielders_agg.top_100 # our target is a boolean variable meaning top 100 or not
ids = outfielders_agg.Player # helper to find the player names

# Train test split (stratify is used to ensure the same proportion of top 100 are in train and test)
X_train, X_test, y_train, y_test, id_train, id_test = train_test_split(
    X, y, ids, test_size=0.20, stratify = y, random_state=42)

id_train = id_train.reset_index().drop("index", axis = 1) # reformatting ids for easier use
id_test = id_test.reset_index().drop("index", axis = 1) # reformatting ids for easier use

log = LogisticRegression(class_weight="balanced") # Logistic regression model
log.fit(X_train, y_train) # Fit to the training data
# y_pred = log.predict(X_test) # get predictions
y_pred_prob = log.predict_proba(X_test) # predicted probabilities of becoming a star
potential = y_pred_prob[:, 1] > 0.85 # Let's say a predicted probability of over 85% is a good start
potential_indexes = [i for i, x in enumerate(potential) if x] # store the indexes of all potential players into an array
for i in range(len(potential_indexes)):
  print(id_test.iloc[potential_indexes[i], 0]) # print future stars (maybe they are already stars)

Marquinhos
Joshua Kimmich
Ismaila Sarr
Andy Delort
Seko Fofana
Adnan Januzaj
Riyad Mahrez
Allan Saint-Maximin
Pierre Højbjerg
Ivan Rakitić
Theo Hernández
Anthony Martial
Jadon Sancho
Marco Reus
Harry Kane
Moussa Dembélé
Rodri
Neymar
Toni Kroos
Raphaël Guerreiro
Gerard Moreno
Paul Pogba
Lassana Diarra
Jordi Alba
Lucas Ocampos
Timo Werner
Wilfried Zaha
Willian
David Silva
Eden Hazard
Josip Iličić
Thiago Alcántara
Saïd Benrahma
Kike Hermoso
Diogo Jota
Amine Gouiri
Hakan Çalhanoğlu
Christopher Nkunku
Edin Džeko


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Where to next?
# print(top_85_outfielders_agg.columns)
# print(top_15_keepers_agg.columns)

# Ranking of top 100 (1st, 10th, unranked)?
# Above top 25, where could they land?
# Transferability (need club name)?
# Team chemistry (combine with KNN)?
# Play-by-play (+ or -)?

                Player        Min       Gls       Ast        PK     PKatt  \
0        Achraf Hakimi  78.454545  0.151515  0.212121  0.000000  0.000000   
1      Ademola Lookman  53.272727  0.136364  0.081818  0.000000  0.009091   
2      Alejandro Balde  37.600000  0.000000  0.000000  0.000000  0.000000   
3   Alessandro Bastoni  76.367521  0.034188  0.042735  0.000000  0.000000   
4       Alexander Isak  56.537037  0.296296  0.046296  0.018519  0.027778   
..                 ...        ...       ...       ...       ...       ...   
67     Vinicius Júnior  58.905983  0.213675  0.119658  0.000000  0.000000   
68     Virgil van Dijk  88.330579  0.107438  0.033058  0.000000  0.000000   
69             Vitinha  27.368421  0.000000  0.052632  0.000000  0.000000   
70      William Saliba  87.011905  0.011905  0.000000  0.000000  0.000000   
71         Xavi Simons  19.142857  0.000000  0.000000  0.000000  0.000000   

          Sh       SoT      CrdY      CrdR  ...        DM        FW        